# Module 1, Section 2: Building Your First Agent


<div align="center">
    <img src="../../images/db_agent.png">
</div>

In Section 1, we built the tool calling loop manually. While educational, that's a lot of tedious code.

In this section, we'll build our database agent using [`create_agent`](https://docs.langchain.com/oss/python/langchain/agents#agents) - a powerful abstraction that:
- Handles the entire tool calling loop automatically
- Adds conversational (i.e. short-term) memory out of the box
- Uses threads for managing separate conversations
- Supports streaming for better UX

By the end, you'll see how `create_agent` replaces ~50 lines of manual code with ~5 lines!

## Setup

Load environment variables:

In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

## 1. Import Tools

**Note on Refactoring:** In Section 1, we defined tools inline for learning purposes. Now that you understand how tools work, we've moved them to the shared `tools/` directory for reuse across multiple sections.

In [ ]:
# Import our shared database tools
from tools.database import (
    get_order_status,
    get_order_items,
    get_product_info,
    get_order_item_price,
)

## 2. Create Your First Agent

Let's replace all that manual loop code with the `create_agent` abstraction which will run the loop for us:
1. Model decides which tool to call (if any)
2. Tool gets executed
3. Result goes back to model
4. Repeat until task is complete

The prebuilt agent handles running the loop described above - you just specify the system prompt and tools.


In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from config import DEFAULT_MODEL

# Initialize model (using workshop default from config.py)
llm = init_chat_model(DEFAULT_MODEL)

# Create agent - THIS REPLACES ALL THE MANUAL LOOP CODE!
agent = create_agent(
    model=llm,
    tools=[get_order_status, get_order_items, get_product_info, get_order_item_price],
    name="db_agent",
    system_prompt="You are a helpful customer support assistant for TechHub.",
)
agent

Notice the benefits:
  - Automatic tool calling loop
  - No manual message management
  - Just ~5 lines of code!

Let's test it with the same query as Section 1.

> Note that we're now passing `messages` formatted as a list of dictionaries instead of the explicit `HumanMessage` class from the last section. Either work, and you can learn more [here](https://docs.langchain.com/oss/python/langchain/messages).


In [ ]:
result = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2024-0123?"}
        ]
    }
)

for message in result["messages"]:
    message.pretty_print()

**Notice:** This works great for single queries! But what happens when we try to ask a follow-up question...

In [ ]:
# Try a follow-up that references the previous query
result = agent.invoke(
    {"messages": [{"role": "user", "content": "When was it shipped?"}]}
)

result["messages"][-1].pretty_print()

⚠️  The agent doesn't remember the previous order! We need memory.

## 3. Add Short-term Memory with Checkpointer

Right now, each agent invocation is independent. Let's add **short-term memory** so the agent can maintain context across multi-turn conversations.

LangChain is built on LangGraph, which uses **checkpointers** to save and restore state. We get this out of the box.

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# Add checkpointer for memory
checkpointer = InMemorySaver()

agent_with_memory = create_agent(
    model=llm,
    tools=[get_order_status, get_order_items, get_product_info, get_order_item_price],
    system_prompt="You are a helpful customer support assistant for TechHub.",
    name="db_agent_with_memory",
    checkpointer=checkpointer,  # This enables memory!
)

When we use checkpointers, we need to pass a `thread_id` when invoking the agent. The `thread_id` acts as a unique identifier for each conversation, allowing the agent to keep separate histories for different users or sessions.

In [ ]:
import uuid

# Create a thread for this conversation - common practice to use a uuid
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

# Turn 1: Ask about an order
print("[Turn 1]")
result = agent_with_memory.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2024-0123?"}
        ]
    },
    config=config,
)
result["messages"][-1].pretty_print()

# Turn 2: Follow-up question (references "it" from previous turn)
print("\n[Turn 2]")
result = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": "When was it shipped?"}]},
    config=config,  # same thread_id
)
result["messages"][-1].pretty_print()

✅ The agent now remembers details from earlier in the conversation

## 4. Thread Separation

Different `thread_id`s create separate conversations with isolated memory:

In [ ]:
# Thread 1: Customer A
config_user1 = {"configurable": {"thread_id": uuid.uuid4()}}
result = agent_with_memory.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2024-0123?"}
        ]
    },
    config=config_user1,
)
print(f"[Customer A]: {result['messages'][-1].content}...\n")

# Thread 2: Customer B (different conversation)
config_user2 = {"configurable": {"thread_id": uuid.uuid4()}}
result = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": "How much is the MacBook Air?"}]},
    config=config_user2,
)
print(f"[Customer B]: {result['messages'][-1].content}...\n")

# Back to Thread 1: Memory is preserved
result = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": "When was it shipped?"}]},
    config=config_user1,
)
print(f"[Customer A follow-up]: {result['messages'][-1].content}...")

### Key Takeaway:
- Checkpointers enable memory across interactions
- Thread IDs separate different conversations
- State persists automatically - no manual state management needed!

## 5. Streaming Responses for Better UX

LLMs can take a while to respond.

**Streaming** shows progress in real-time, dramatically improving user experience. To stream responses token-by-token - just change `.invoke()` to `.stream()`:

### Streaming Agent Steps

In [ ]:
# Stream agent progress with stream_mode="updates"
print("Streaming agent steps:\n")

for chunk in agent.stream(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2024-0125?"}
        ]
    },
    stream_mode="updates",
):
    for node_name, data in chunk.items():
        print(f"Step: {node_name}")
        if "messages" in data:
            message = data["messages"][-1]
            if hasattr(message, "tool_calls") and message.tool_calls:
                print(f"   Tool call: {message.tool_calls[0]['name']}")
            elif hasattr(message, "content"):
                print(f"   Content: {message.content}")
        print()

### Streaming LLM Tokens

In [ ]:
print("Streaming response:\n")

for message_chunk, metadata in agent.stream(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2024-0125?"}
        ]
    },
    stream_mode="messages",
):
    # only print content from the model node
    if metadata.get("langgraph_node") == "model":
        # Get text from content blocks
        for block in message_chunk.content_blocks:
            if block.get("type") == "text" and block.get("text"):
                print(block.get("text"), end="", flush=True)

Key Takeaway:
- `stream_mode="updates"` - See each agent step (useful for debugging)
- `stream_mode="messages"` - Stream LLM tokens (ChatGPT-like UX)
- Streaming is built-in - no extra setup required!

> **Note:** You can control what types of information are streamed (e.g., messages, tool calls, etc.) by setting the `stream_mode` parameter. This allows fine-grained control over what you receive in real-time. See the [LangGraph streaming docs](https://docs.langchain.com/oss/python/langgraph/streaming) for details.


## 6. Review the agent traces in LangSmith!